# Mistplay Fraud Demo – Train & Register Model

This notebook uses Feature Store to build a training set, trains a fraud classifier, tracks metrics with MLflow, and registers the model.

In [ ]:
import mlflow
import mlflow.sklearn
import pandas as pd
from databricks.feature_store import FeatureStoreClient, FeatureLookup
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

DB_NAME = "mistplay_fraud_demo"
MODEL_NAME = "mistplay_fraud_model"

fs = FeatureStoreClient()

training_base = spark.table(f"{DB_NAME}.training_base")

lookups = [
    FeatureLookup(table_name=f"{DB_NAME}.account_features", lookup_key="account_id"),
    FeatureLookup(table_name=f"{DB_NAME}.device_features", lookup_key="device_id"),
]

training_set = fs.create_training_set(
    training_base,
    feature_lookups=lookups,
    label="is_fraud_label",
    exclude_columns=["account_id", "device_id"],
)

training_df = training_set.load_df().toPandas()

label_col = "is_fraud_label"
categorical_cols = [
    "country",
    "platform",
    "marketing_channel",
    "device_type",
    "os_version",
]

numeric_cols = [
    c for c in training_df.columns if c not in categorical_cols + [label_col]
]

X = training_df.drop(columns=[label_col])
y = training_df[label_col]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

preprocess = ColumnTransformer(
    transformers=[
        ("categorical", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("numeric", StandardScaler(), numeric_cols),
    ]
)

model = LogisticRegression(max_iter=200, class_weight="balanced")

pipeline = Pipeline(steps=[("preprocess", preprocess), ("model", model)])

with mlflow.start_run() as run:
    pipeline.fit(X_train, y_train)
    preds = pipeline.predict_proba(X_test)[:, 1]

    roc_auc = roc_auc_score(y_test, preds)
    pr_auc = average_precision_score(y_test, preds)

    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("pr_auc", pr_auc)

    fs.log_model(
        model=pipeline,
        artifact_path="model",
        flavor=mlflow.sklearn,
        training_set=training_set,
        registered_model_name=MODEL_NAME,
    )

print(f"Registered model: {MODEL_NAME}")